Install the required python packages if not already present.

In [7]:
# !pip install gradio
# !pip install langchain
# !pip install openai

In [3]:
# import required packages
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
import os
import openai

Before executing the next cells, make sure that a valid Azure subscription is available and an instance of Azure OpenAI services is created. 

Then add the credentials for the Azure API calls in the following manner:

```
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = #your model endpoint
os.environ["OPENAI_API_KEY"] = #your api key
```

In [4]:
# define a class with all required methods
class App:
    def __init__(self) -> None:
        self.qa = None
    
    def load_doc(self, file_obj):
        '''Load PDF document and parse by page'''
        loader = PyPDFLoader(file_obj.name)
        pages = loader.load_and_split()
        return pages
    
    def upload_file(self, file_obj):
        '''Handle the upload request'''
        pages = self.load_doc(file_obj)
        # initialize embeddings
        embeddings = OpenAIEmbeddings(chunk_size=1)
        # generate embeddings for the uploaded doc
        docsearch = FAISS.from_documents(pages, embeddings)
        # params
        temperature = 0
        deployment_name = "text-davinci-003"
        max_tokens = 500
        
        llm = AzureOpenAI(temperature=temperature, deployment_name=deployment_name, max_tokens=max_tokens)

        # prepare the query answering object
        self.qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=docsearch.as_retriever(search_type='mmr'))
        # make the subsequent components of the UI visible
        return gr.Textbox().update(visible=True), gr.Checkbox.update(visible=True), gr.Button.update(visible=True), gr.Textbox.update(visible=True)

    def response_optimizer(self, query_response):
        '''Optimize the first response from the model to generate a more interpretable response'''
        prompt = f'''Optimize the following text and rephrase it in simpler terms to make it easier to read and interpret. Use bullet points wherever necessary.
        
        # Text:
        {query_response}

        # Optimized Text:'''

        # langchain is not required for this purpose
        openai.api_key = os.getenv('OPENAI_API_KEY')
        openai.api_base =  os.getenv('OPENAI_API_BASE')
        openai.api_type = os.getenv('OPENAI_API_TYPE')
        openai.api_version = os.getenv('OPENAI_API_VERSION')
        response = openai.Completion.create(engine='text-davinci-003', prompt=prompt, max_tokens= 1000, temperature=0)
        response = response['choices'][0]['text']
        return response         

    def answer_query(self, query):
        '''Handle query request and return response'''
        query_response = self.qa.run(query)
        optimized_response = self.response_optimizer(query_response)
        return optimized_response
        

In [5]:
# initialize the function
app = App()

In [6]:
demo = gr.Blocks(theme=gr.themes.Monochrome())

with demo:
    with gr.Row():
        
        file1 = gr.File(info = "upload the policy pdf file", file_count='single', file_types=['.pdf'])    
    upload_btn = gr.Button("Upload")
        
    with gr.Row():
            with gr.Column():
                query_text = gr.Textbox(label="Enter your question", interactive=True, visible=False)
                process_btn = gr.Button("Get Answer", visible=False)
            with gr.Column():
                response_text = gr.Textbox(label="Response:", visible=False)
    
    
    upload_btn.click(app.upload_file, inputs=[file1], outputs = [query_text, process_btn, response_text])
    process_btn.click(app.answer_query, inputs=[query_text], outputs = [response_text])

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

On Successful run of the above cells, we would be able to observe the following user-interface:
![img 1](./resources/1.jpg)

We can then upload any pdf file of our choice and wait till the query area appears:
![img 2](./resources/2.jpg)

![img 3](./resources/3.jpg)

We can now put our query in the text box and click on the "Get Answer" button to trigger the API call to the model:
![img 4](./resources/4.jpg)

The model takes a few seconds to process and the result appears on the right-hand side:
![img 5](./resources/5.jpg)

Let's try another query:
![img 6](./resources/6.jpg)